# Exercise 04. Enrichment and transformations

In [1]:
import pandas as pd
import numpy as np
import requests

## Read the JSON file that you saved in ex02

In [2]:
pd.options.display.float_format = '{:.2f}'.format
df = pd.read_json('../data/auto.json', orient='records')
df

,CarNumber,Refund,Fines,Make,Model
0,Y163O8161RUS,2,3200.00,Ford,Focus
1,E432XX77RUS,1,6500.00,Toyota,Camry
2,7184TT36RUS,1,2100.00,Ford,Focus
3,X582HE161RUS,2,2000.00,Ford,Focus
4,92918M178RUS,1,5700.00,Ford,Focus
...,...,...,...,...,...
720,Y163O8161RUS,2,1600.00,Ford,Focus
721,M0309X197RUS,1,22300.00,Ford,Focus
722,O673E8197RUS,2,600.00,Ford,Focus
723,8610T8154RUS,1,2000.00,Ford,Focus


## Enrich the dataframe using a sample from that dataframe

In [3]:
sample = df.sample(n=200, random_state=21)
sample

,CarNumber,Refund,Fines,Make,Model
445,M0299X197RUS,2,19200.00,Ford,Focus
22,83298C154RUS,2,8594.60,Ford,Focus
93,H957HY161RUS,1,2000.00,Ford,Focus
173,T941CC96RUS,1,2000.00,Ford,Focus
697,H966HY161RUS,1,500.00,Ford,Focus
...,...,...,...,...,...
14,8182XX154RUS,1,200.00,Ford,Focus
623,X796TH96RUS,1,500.00,Ford,Focus
498,T011MY163RUS,2,4000.00,Ford,Focus
536,T341CC96RUS,2,1000.00,Volkswagen,Passat


In [4]:
concat_rows = pd.concat(objs=(df, sample), ignore_index=True)
concat_rows

,CarNumber,Refund,Fines,Make,Model
0,Y163O8161RUS,2,3200.00,Ford,Focus
1,E432XX77RUS,1,6500.00,Toyota,Camry
2,7184TT36RUS,1,2100.00,Ford,Focus
3,X582HE161RUS,2,2000.00,Ford,Focus
4,92918M178RUS,1,5700.00,Ford,Focus
...,...,...,...,...,...
920,8182XX154RUS,1,200.00,Ford,Focus
921,X796TH96RUS,1,500.00,Ford,Focus
922,T011MY163RUS,2,4000.00,Ford,Focus
923,T341CC96RUS,2,1000.00,Volkswagen,Passat


## Enrich the `concat_rows` dataframe with a new column containing generated data

In [5]:
np.random.seed(21)
Year = pd.Series(data=np.random.randint(1980, 2019, size=len(concat_rows)), name='Year')
fines = pd.concat(objs=(concat_rows, Year), axis='columns')
fines

,CarNumber,Refund,Fines,Make,Model,Year
0,Y163O8161RUS,2,3200.00,Ford,Focus,1989
1,E432XX77RUS,1,6500.00,Toyota,Camry,1995
2,7184TT36RUS,1,2100.00,Ford,Focus,1984
3,X582HE161RUS,2,2000.00,Ford,Focus,2015
4,92918M178RUS,1,5700.00,Ford,Focus,2014
...,...,...,...,...,...,...
920,8182XX154RUS,1,200.00,Ford,Focus,1996
921,X796TH96RUS,1,500.00,Ford,Focus,2002
922,T011MY163RUS,2,4000.00,Ford,Focus,1996
923,T341CC96RUS,2,1000.00,Volkswagen,Passat,2012


## Enrich the dataframe with data from another dataframe

In [6]:
surnames = pd.read_json('../data/surname.json', orient='values')
surnames.columns = surnames.iloc[0]
surnames.drop(0, inplace=True)
surnames


,NAME,COUNT,RANK
1,ADAMS,427865,42
2,ALLEN,482607,33
3,ALVAREZ,233983,92
4,ANDERSON,784404,15
5,BAILEY,277845,72
...,...,...,...
96,WILLIAMS,1625252,3
97,WILSON,801882,14
98,WOOD,250715,84
99,WRIGHT,458980,35


In [7]:
car_numbers = concat_rows.drop_duplicates('CarNumber')['CarNumber']
car_numbers = car_numbers.to_frame(name='CarNumber').reset_index(drop=True)
car_numbers

,CarNumber
0,Y163O8161RUS
1,E432XX77RUS
2,7184TT36RUS
3,X582HE161RUS
4,92918M178RUS
...,...
526,O136HO197RUS
527,O22097197RUS
528,M0309X197RUS
529,O673E8197RUS


In [8]:
surnames = surnames['NAME'].sample(n=len(car_numbers), random_state=21, replace=True)
surnames = surnames.to_frame(name='SURNAME').reset_index(drop=True)
surnames

,SURNAME
0,RICHARDSON
1,ROSS
2,MORGAN
3,BAILEY
4,LOPEZ
...,...
526,CAMPBELL
527,HALL
528,BAKER
529,DIAZ


In [9]:
owners = pd.concat(objs=(car_numbers, surnames), axis='columns')
owners

,CarNumber,SURNAME
0,Y163O8161RUS,RICHARDSON
1,E432XX77RUS,ROSS
2,7184TT36RUS,MORGAN
3,X582HE161RUS,BAILEY
4,92918M178RUS,LOPEZ
...,...,...
526,O136HO197RUS,CAMPBELL
527,O22097197RUS,HALL
528,M0309X197RUS,BAKER
529,O673E8197RUS,DIAZ


In [10]:
new_obs = pd.DataFrame({'CarNumber': ['A000AA50', 'B111BB50', 'C222CC50', 'X999XX50', 'A123BC50'],
                        'Refund': [1, 2, 1, 2, 1],
                        'Fines': [3000, 4000, 5000, 6000, 7000],
                        'Make': ['Toyota', 'Toyota', 'Toyota', 'Toyota', 'Toyota'],
                        'Model': ['Camry', 'Camry', 'Camry', 'Camry', 'Camry'],
                        'Year': [2007, 2007, 2007, 2007, 2007]})
fines = pd.concat(objs=(fines, new_obs), ignore_index=True)
fines

,CarNumber,Refund,Fines,Make,Model,Year
0,Y163O8161RUS,2,3200.00,Ford,Focus,1989
1,E432XX77RUS,1,6500.00,Toyota,Camry,1995
2,7184TT36RUS,1,2100.00,Ford,Focus,1984
3,X582HE161RUS,2,2000.00,Ford,Focus,2015
4,92918M178RUS,1,5700.00,Ford,Focus,2014
...,...,...,...,...,...,...
925,A000AA50,1,3000.00,Toyota,Camry,2007
926,B111BB50,2,4000.00,Toyota,Camry,2007
927,C222CC50,1,5000.00,Toyota,Camry,2007
928,X999XX50,2,6000.00,Toyota,Camry,2007


In [11]:
owners.drop(owners.tail(20).index, inplace=True)
new_obs = pd.DataFrame({'CarNumber': ['Q666QQ70', 'W000WW70', 'R888RR70'],
                        'SURNAME': ['HILL', 'MORGAN', 'MARTIN']})
owners = pd.concat(objs=(owners, new_obs), ignore_index=True)
owners

,CarNumber,SURNAME
0,Y163O8161RUS,RICHARDSON
1,E432XX77RUS,ROSS
2,7184TT36RUS,MORGAN
3,X582HE161RUS,BAILEY
4,92918M178RUS,LOPEZ
...,...,...
509,O50197197RUS,WRIGHT
510,7608EE777RUS,HILL
511,Q666QQ70,HILL
512,W000WW70,MORGAN


In [12]:
inner_df = pd.merge(fines, owners, how='inner')
inner_df

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
0,Y163O8161RUS,2,3200.00,Ford,Focus,1989,RICHARDSON
1,E432XX77RUS,1,6500.00,Toyota,Camry,1995,ROSS
2,7184TT36RUS,1,2100.00,Ford,Focus,1984,MORGAN
3,X582HE161RUS,2,2000.00,Ford,Focus,2015,BAILEY
4,92918M178RUS,1,5700.00,Ford,Focus,2014,LOPEZ
...,...,...,...,...,...,...,...
894,8182XX154RUS,1,200.00,Ford,Focus,1996,SMITH
895,X796TH96RUS,1,500.00,Ford,Focus,2002,WATSON
896,T011MY163RUS,2,4000.00,Ford,Focus,1996,SANDERS
897,T341CC96RUS,2,1000.00,Volkswagen,Passat,2012,PEREZ


In [13]:
outer_df = pd.merge(fines, owners, how='outer')
outer_df

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
0,704687163RUS,2.00,1400.00,Ford,Focus,2014.00,ADAMS
1,704787163RUS,2.00,2800.00,Ford,Focus,2005.00,MORGAN
2,704987163RUS,2.00,8594.60,Ford,Focus,2014.00,MITCHELL
3,705287163RUS,2.00,2000.00,Ford,Focus,1990.00,GOMEZ
4,705387163RUS,2.00,700.00,Ford,Focus,2005.00,STEWART
...,...,...,...,...,...,...,...
928,Y973O8197RUS,2.00,8594.60,Ford,Focus,2005.00,YOUNG
929,Y973O8197RUS,1.00,34800.00,Ford,Focus,2013.00,YOUNG
930,Y973O8197RUS,1.00,69600.00,Ford,Focus,1989.00,YOUNG
931,Y973O8197RUS,1.00,34800.00,Ford,Focus,2009.00,YOUNG


In [14]:
left_df = pd.merge(fines, owners, how='left')
left_df

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
0,Y163O8161RUS,2,3200.00,Ford,Focus,1989,RICHARDSON
1,E432XX77RUS,1,6500.00,Toyota,Camry,1995,ROSS
2,7184TT36RUS,1,2100.00,Ford,Focus,1984,MORGAN
3,X582HE161RUS,2,2000.00,Ford,Focus,2015,BAILEY
4,92918M178RUS,1,5700.00,Ford,Focus,2014,LOPEZ
...,...,...,...,...,...,...,...
925,A000AA50,1,3000.00,Toyota,Camry,2007,NaN
926,B111BB50,2,4000.00,Toyota,Camry,2007,NaN
927,C222CC50,1,5000.00,Toyota,Camry,2007,NaN
928,X999XX50,2,6000.00,Toyota,Camry,2007,NaN


In [15]:
right_df = pd.merge(fines, owners, how='right')
right_df

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
0,Y163O8161RUS,2.00,3200.00,Ford,Focus,1989.00,RICHARDSON
1,Y163O8161RUS,2.00,1600.00,Ford,Focus,1999.00,RICHARDSON
2,E432XX77RUS,1.00,6500.00,Toyota,Camry,1995.00,ROSS
3,E432XX77RUS,2.00,13000.00,Toyota,Camry,1992.00,ROSS
4,7184TT36RUS,1.00,2100.00,Ford,Focus,1984.00,MORGAN
...,...,...,...,...,...,...,...
897,7608EE777RUS,1.00,4000.00,Skoda,Octavia,2013.00,HILL
898,7608EE777RUS,1.00,4000.00,Skoda,Octavia,1987.00,HILL
899,Q666QQ70,NaN,NaN,NaN,NaN,NaN,HILL
900,W000WW70,NaN,NaN,NaN,NaN,NaN,MORGAN


## Create a pivot table from the `fines` dataframe. It should look like this (the values are the sums of the fines), but with all the years. The values may be different for you

In [16]:
pd.pivot_table(fines, columns='Year', values='Fines', index=['Make', 'Model'], aggfunc={'Fines': np.sum})

/tmp/ipykernel_294256/588614432.py:1: FutureWarning: The provided callable <function sum at 0x7fb703919760> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pd.pivot_table(fines, columns='Year', values='Fines', index=['Make', 'Model'], aggfunc={'Fines': np.sum})


Year                   1980      1981      1982      1983      1984      1985  \
Make       Model                                                                
Ford       Focus   89194.60 266783.80 107283.80 147289.20 106000.00 307494.60   
           Mondeo       NaN       NaN  46200.00       NaN       NaN       NaN   
Skoda      Octavia 13794.60   1900.00   8894.60       NaN   1300.00 153594.60   
Toyota     Camry   12000.00       NaN   1000.00   8594.60   1000.00       NaN   
           Corolla      NaN   6800.00       NaN  12800.00       NaN   4400.00   
Volkswagen Golf    20800.00   8594.60   5000.00    200.00       NaN 168000.00   
           Jetta        NaN   1000.00       NaN       NaN       NaN   9000.00   
           Passat    900.00  12500.00       NaN   1100.00   8594.60       NaN   
           Touareg      NaN       NaN       NaN       NaN       NaN       NaN   

Year                   1986     1987     1988      1989  ...      2009  \
Make       Model                                         ...             
Ford       Focus   69700.00 98189.20 69667.60 200889.20  ... 159894.60   
           Mondeo       NaN      NaN      NaN       NaN  ...       NaN   
Skoda      Octavia      NaN  6000.00  5100.00   8594.60  ...       NaN   
Toyota     Camry   19800.00      NaN      NaN    800.00  ...       NaN   
           Corolla      NaN 54300.00      NaN   7800.00  ...   8594.60   
Volkswagen Golf         NaN   300.00      NaN    300.00  ...       NaN   
           Jetta        NaN      NaN 46000.00   4000.00  ...       NaN   
           Passat  16000.00  2000.00  8594.60       NaN  ...   3200.00   
           Touareg      NaN      NaN      NaN       NaN  ...   5800.00   

Year                   2010      2011      2012      2013     2014      2015  \
Make       Model                                                               
Ford       Focus   96000.00 117194.60 152989.20 297378.40 90378.40 172700.00   
           Mondeo       NaN       NaN       NaN  41100.00      NaN       NaN   
Skoda      Octavia  3000.00   3000.00   1700.00  11800.00 18900.00  16394.60   
Toyota     Camry   22400.00       NaN   7500.00       NaN      NaN       NaN   
           Corolla  6000.00   3400.00       NaN       NaN      NaN  30300.00   
Volkswagen Golf         NaN       NaN       NaN       NaN 13900.00   4600.00   
           Jetta        NaN       NaN       NaN       NaN      NaN       NaN   
           Passat   9500.00       NaN   1000.00   1600.00 15000.00       NaN   
           Touareg      NaN       NaN       NaN       NaN      NaN       NaN   

Year                    2016      2017      2018  
Make       Model                                  
Ford       Focus   110789.20 114089.20 107800.00  
           Mondeo        NaN   8600.00       NaN  
Skoda      Octavia  35700.00   2400.00 153200.00  
Toyota     Camry      800.00       NaN       NaN  
           Corolla       NaN   6900.00       NaN  
Volkswagen Golf          NaN       NaN   1000.00  
           Jetta         NaN       NaN       NaN  
           Passat        NaN       NaN       NaN  
           Touareg       NaN       NaN       NaN  

[9 rows x 39 columns]

## Save both the `fines` and `owners` dataframes to CSV files without an index

In [17]:
fines.to_csv('../data/fines.csv', index=False)
owners.to_csv('../data/owners.csv', index=False)